# Joint Analysis of HiPSCats

Here we will demonstrate the LSDB capabilities of analyzing multiple catalogs. LSDB comes equipped with its own nearest-neighbor crossmatching routine. 

### First we start with opening two catalogs from a cloud almanac

In [1]:
from hipscat.inspection.almanac import Almanac
from hipscat.ADASS_Tutorial.credentials import read_storage_options

HIPS_DIR = "abfs:///hipscat/catalogs/almanac"
storage_options = read_storage_options()

cloud_almanac = Almanac(dirs=[HIPS_DIR], storage_options=storage_options)
for catalog in cloud_almanac.catalogs():
    print(catalog)

gaia
ztf_dr14
ztf_source


#### Reading a lsdb.Catalog
Can be read from local disk pathway or from cloud bucket

In [2]:
import lsdb
path_to_cloud_catalog = "abfs:///hipscat/catalogs/gaia_dr3"
catalog = lsdb.read_hipscat(path_to_cloud_catalog, storage_options=storage_options)

In [3]:
ztf = lsdb.from_almanac("ztf_dr14", cloud_almanac)

In [4]:
#look at ztf to choose which columns we want
ztf.dtypes

ps1_objid            int64
ra                 float64
dec                float64
ps1_gMeanPSFMag    float64
ps1_rMeanPSFMag    float64
ps1_iMeanPSFMag    float64
nobs_g               int32
nobs_r               int32
nobs_i               int32
mean_mag_g         float64
mean_mag_r         float64
mean_mag_i         float64
Norder               int32
Dir                  int32
Npix                 int32
dtype: object

In [5]:

gaia = lsdb.from_almanac("gaia", cloud_almanac, columns=['ra', 'dec', 'pmra', 'pmdec'])
ztf = lsdb.from_almanac("ztf_dr14", cloud_almanac, columns=['ra', 'dec', 'ps1_objid', 'nobs_g', 'nobs_r', 'nobs_i', 'mean_mag_g', 'mean_mag_r', 'mean_mag_i'])

In [6]:
gaia

,ra,dec,pmra,pmdec
npartitions=3933,,,,
,float64,float64,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [7]:
ztf

,ra,dec,ps1_objid,nobs_g,nobs_r,nobs_i,mean_mag_g,mean_mag_r,mean_mag_i
npartitions=2352,,,,,,,,,
,float64,float64,int64,int32,int32,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [8]:
#open up our distributed client
client = lsdb.lsdb_client(dask_on_ray=True, num_workers=8)

2023-11-05 06:21:00,142	WARNING services.py:1889 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67100672 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=7.65gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-11-05 06:21:01,306	INFO worker.py:1642 -- Started a local Ray instance.


## Cross-matching

Let's cull our gaia catalog by performing a conesearch, then cross-match it with ztf.

What our cross_match routine does is performs a nearest-neighbor (1) with a distance threshold of 0.1 degrees

In [9]:
%%time
xmatch = gaia.cone_search(
    ra=30,
    dec=30,
    radius=5,
).crossmatch(
    ztf,
    n_neighbors=1,
    d_thresh=0.1, #degrees
)

CPU times: user 106 ms, sys: 3.3 ms, total: 109 ms
Wall time: 105 ms


In [10]:
%%time
xmatch.compute()

CPU times: user 1.33 s, sys: 360 ms, total: 1.69 s
Wall time: 13.5 s


,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia,ra_ztf_dr14,dec_ztf_dr14,ps1_objid_ztf_dr14,nobs_g_ztf_dr14,nobs_r_ztf_dr14,nobs_i_ztf_dr14,mean_mag_g_ztf_dr14,mean_mag_r_ztf_dr14,mean_mag_i_ztf_dr14,_DIST
_hipscat_index,,,,,,,,,,,,,,
209131110217547776,31.553529,25.194838,1.479070,-3.852670,31.553521,25.194859,138230315535194404,71,502,0,19.177027,18.666689,NaN,0.000022
209134163939295232,31.603199,25.210997,-2.706803,-2.627695,31.603185,25.211023,138250316031943809,516,656,44,16.746884,16.163387,15.976430,0.000028
209134176824197120,31.600501,25.236400,1.783758,-1.251412,31.600496,25.236430,138280316004614335,352,567,38,20.407255,19.776944,19.554832,0.000031
209134541896417280,31.594369,25.204140,NaN,NaN,31.594343,25.204133,138240315943505582,63,616,0,19.681671,18.375660,NaN,0.000025
209134215478902784,31.656121,25.220908,-22.330928,-19.116853,31.656084,25.220917,138260316561415711,523,669,44,13.761570,13.218300,13.033317,0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661636185780649984,30.232593,34.982756,-1.569892,-1.575927,30.232583,34.982784,149970302325819951,419,643,45,17.043033,16.502429,16.314149,0.000030
661636190075617280,30.251423,34.990097,-0.358637,-0.079130,30.251419,34.990117,149980302514168764,383,613,40,19.492228,18.849787,18.610821,0.000021
661636215845421056,30.256577,34.987627,0.648915,-2.557683,30.256584,34.987659,149980302565615798,367,612,39,19.671928,18.766456,18.453084,0.000033


### Performing a computation based off the xmatch result

Let's create a column based on the columns we selected from each catalog, and then reduce our result based on that column. Say, I want high-proper motion stars with `n_obs` greater than 50 in each ZTF photometric band. 
* I will assign a `pm` column 
* then query based off the `pm` column, and the `nobs_filter_ztf_dr14` columns

In [11]:
%%time
import numpy as np

gaia.cone_search(
    ra=30,
    dec=30,
    radius=5,
).crossmatch(
    ztf,
    n_neighbors=1,
    d_thresh=0.1, #degrees
).assign(
    pm=lambda x: np.sqrt(x['pmra_gaia']**2 + x['pmdec_gaia']**2),
).query(
    "pm > 100 and `nobs_g_ztf_dr14` > 50 and `nobs_r_ztf_dr14` > 50 and `nobs_i_ztf_dr14` > 50"
).compute()

CPU times: user 1.52 s, sys: 272 ms, total: 1.79 s
Wall time: 6.83 s


,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia,ra_ztf_dr14,dec_ztf_dr14,ps1_objid_ztf_dr14,nobs_g_ztf_dr14,nobs_r_ztf_dr14,nobs_i_ztf_dr14,mean_mag_g_ztf_dr14,mean_mag_r_ztf_dr14,mean_mag_i_ztf_dr14,_DIST,pm
_hipscat_index,,,,,,,,,,,,,,,
209325917049192448,31.884618,25.374571,92.664009,-85.708535,31.884717,25.374499,138450318844320239,331,594,53,20.558525,19.716055,19.450653,0.000114,126.224291
209388761010667520,32.186842,25.607346,352.423213,57.478758,32.185327,25.607863,138720321852909994,528,664,57,18.266217,17.733780,17.590956,0.001461,357.079724
209410635279106048,32.227512,25.777228,104.097418,-59.803741,32.227613,25.777195,138930322273043358,552,705,63,17.371850,15.867299,14.567357,0.000097,120.053154
210468472839208960,29.181819,25.475167,-50.540693,102.676883,29.181763,25.475274,138570291818090730,557,702,62,16.115516,14.603947,13.420323,0.000119,114.441705
210481430755540992,29.608513,25.512488,177.541997,-397.514861,29.604899,25.509625,138610296048712137,261,561,54,20.932020,19.848311,19.464607,0.004340,435.361029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651624350725701632,35.238635,31.736905,108.596308,4.818504,35.238750,31.736925,146080352385474883,834,1162,60,18.510647,17.021626,15.818761,0.000100,108.703156
651626309230788608,35.333964,31.778334,99.119856,-32.611290,35.334070,31.778315,146130353338314680,537,1295,107,20.667411,19.103013,17.510154,0.000093,104.346740
652984223270961152,33.538141,32.646703,91.814173,-46.997280,33.538263,32.646675,147170335380066741,420,649,57,16.130581,14.750207,14.073177,0.000107,103.143524


In [12]:
client.shutdown()